In [1]:
#!/usr/bin/env python
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np, os, sys
from tqdm.notebook import tqdm    
from manipulations import get_classes, get_classes_from_header, get_Fs_from_header, load_challenge_data
from sklearn.model_selection import KFold, StratifiedKFold

In [2]:
N_CLASS = 27

In [3]:
MAX_RR = 750 # 60 beats/min => 60 beats/60 s ==> beat/1s ==> 500 samples / beat

In [4]:
from manipulations import get_abbr, get_name
from global_vars import labels, Dx_map, Dx_map_unscored
first_idx = {scored_code: None for scored_code in list(Dx_map['SNOMED CT Code'])}
first_idx_unscored = {unscored_code: None for unscored_code in list(Dx_map_unscored['SNOMED CT Code'])}

In [5]:
from global_vars import labels, normal_class, equivalent_mapping
normal_idx = np.argwhere(labels==int(normal_class))
def get_scored_class(code, labels):
    return [1 if label in code else 0 for label in labels]

# On y va

In [6]:
Signals_train = np.load('../saved/Signals_train_stratified.npy')
Signals_test = np.load('../saved/Signals_test_stratified.npy')

In [7]:
import pickle

with open('../saved/Data_labels_train_stratified.pkl', 'rb') as Data_labels_train_file:
    Data_labels_train = pickle.load(Data_labels_train_file)

with open('../saved/Data_labels_test_stratified.pkl', 'rb') as Data_labels_test_file:
    Data_labels_test = pickle.load(Data_labels_test_file)

In [8]:
"""
Encode old label (multi-hot encoding) to new label (number) with a threshold (>=80-percentile length)
"""
Code_label_train = [get_scored_class(Data_labels_train[i], labels) for i in range(len(Data_labels_train)) ]


# key: multi-hot encoding label in str
# val: idx
labels_dict = {}
for idx, data_label in enumerate(Code_label_train):
    key = ''.join([str(l) for l in data_label])
    if key not in labels_dict:
        labels_dict[key] = [idx]
    else:
        labels_dict[key].append(idx)

# for each label (multi-hot encoding), their length
labels_dict_len = []
labels_dict_key = []
for key in labels_dict.keys():
    labels_dict_len.append(len(labels_dict[key]))
    labels_dict_key.append(key)
labels_dict_len = np.array(labels_dict_len)
labels_dict_key = np.array(labels_dict_key)

threshold = np.percentile(labels_dict_len, 80)

labels_dict_len_threshold = labels_dict_len[labels_dict_len>threshold]
labels_dict_key_threshold = labels_dict_key[labels_dict_len>threshold]

# old label (multi-hot encoding) => new label (number)
new_index_dict = {}
for i in range(len(labels_dict_key_threshold)):
    new_index_dict[labels_dict_key_threshold[i]] = i

# change label to new label
Code_label_train_new = np.zeros((len(Code_label_train),), dtype = int)
special_class = -1
for key, vals in labels_dict.items():
    for val in vals:
        if key not in labels_dict_key_threshold:
            Code_label_train_new[val] = special_class
        else:
            Code_label_train_new[val] = new_index_dict[key]
        

# distribution of classes in the dataset 
label_to_count = {}
for label in Code_label_train_new:
    if label in label_to_count:
        label_to_count[label] += 1
    else:
        label_to_count[label] = 1

# weight for each sample
sample_weights = [1.0 / label_to_count[label]
           for label in Code_label_train_new]
import torch
sample_weights = torch.DoubleTensor(sample_weights)

In [10]:
assert len(sample_weights) ==  len(Data_labels_train)
assert len(Signals_train) == len(Data_labels_train)

In [74]:
run_name = 'ECGResNet50_balanced_sampleweights_eqvclasses_no_posweight'

In [75]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/{}'.format(run_name))


In [54]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

In [55]:
import torch
from torch import nn

In [56]:

def weighted_binary_cross_entropy2(sigmoid_x, y, weighted_matrix, weight=None, reduction=None):
    """
    Aha this is correct!
    sigmoid_x = nn.Sigmoid()(x)
    Args:
        sigmoid_x: predicted probability of size [N,C], N sample and C Class. Eg. Must be in range of [0,1]
        targets: true value, one-hot-like vector of size [N,C]
        pos_weight: Weight for postive sample
    """
    if not (y.size() == sigmoid_x.size()):
        raise ValueError("Target size ({}) must be the same as input size ({})".format(y.size(), sigmoid_x.size()))
   
    #print("y.size(), sigmoid_x.size()", y.size(), sigmoid_x.size())
    sigmoid_x = torch.clamp(sigmoid_x,min=1e-7,max=1-1e-7) 
    loss = - torch.matmul(y*sigmoid_x.log() + (1-y)*(1-sigmoid_x).log(), weighted_matrix)
    
    if weight is not None:
        loss = loss * weight
        
    if reduction is None:
        return loss
    elif reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    return None
    
class WeightedBCELoss(nn.Module):
    def __init__(self, weights, PosWeightIsDynamic= False, WeightIsDynamic= False, 
                 reduction='mean'):
        """
        Args:
            pos_weight = Weight for postive samples. Size [1,C]
            weight = Weight for Each class. Size [1,C]
            PosWeightIsDynamic: If True, the pos_weight is computed on each batch. If pos_weight is None, then it remains None.
            WeightIsDynamic: If True, the weight is computed on each batch. If weight is None, then it remains None.
        """
        super().__init__()

        self.register_buffer('weights', weights)
        self.reduction = reduction
        self.PosWeightIsDynamic = PosWeightIsDynamic

    def forward(self, input, target):
        if self.PosWeightIsDynamic:
            positive_counts = target.sum(dim=0)
            nBatch = len(target)
            self.pos_weight = (nBatch - positive_counts)/(positive_counts +1e-5)


        return weighted_binary_cross_entropy2(input, target,
                                             weighted_matrix=self.weights,
                                             reduction=self.reduction)

In [57]:
Signal_12leads_train = np.transpose(Signals_train, (1,0,2))
Signal_12leads_test= np.transpose(Signals_test, (1,0,2))

In [58]:
from torch.utils.data import Dataset
from torchvision import transforms

import os
class SignalDataset(Dataset):

    def __init__(self, signals, labels):
        self.signals = signals
        self.labels = labels

    def __len__(self):
        return len(self.signals)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample =(torch.cat([torch.Tensor(np.array([self.signals[channel,idx]]).transpose()) for channel in range(12)], axis=1), 
                  torch.Tensor(self.labels[idx]))

        return sample
    
signal_datasets_train = SignalDataset(Signal_12leads_train, np.array(Data_labels_train)[:,:N_CLASS])
signal_datasets_test = SignalDataset(Signal_12leads_test, np.array(Data_labels_test)[:,:N_CLASS])


In [59]:
assert np.array(Data_labels_train).shape[1] == N_CLASS

In [60]:
signal_datasets_train[0][0].shape

torch.Size([750, 12])

In [61]:
from global_vars import weights

In [62]:
from evaluation.evaluate_12ECG_score import compute_modified_confusion_matrix, compute_challenge_metric

In [63]:
def compute_score(labels, outputs, weights, normal_index=normal_idx):
    num_recordings, num_classes = np.shape(labels)
    # Compute the observed score.
    A = compute_modified_confusion_matrix(labels, outputs)
    observed_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the correct label(s).
    correct_outputs = labels
    A = compute_modified_confusion_matrix(labels, correct_outputs)
    correct_score = np.nansum(weights * A)

    # Compute the score for the model that always chooses the normal class.
    inactive_outputs = np.zeros((num_recordings, num_classes), dtype=np.bool)
    inactive_outputs[:, normal_index] = 1
    A = compute_modified_confusion_matrix(labels, inactive_outputs)
    inactive_score = np.nansum(weights * A)

    if correct_score != inactive_score:
        normalized_score = float(observed_score - inactive_score) / float(correct_score - inactive_score)
    else:
        normalized_score = float('nan')

    return normalized_score

In [64]:
def add_pr_curve_tensorboard(class_index, test_probs, test_preds, names, global_step, prefix):
    '''
    Takes in a "class_index" from 0 to 9 and plots the corresponding
    precision-recall curve
    '''

    writer.add_pr_curve(prefix + '_' + names[class_index],
                        test_preds[:,class_index],
                        test_probs[:,class_index],
                        global_step=global_step,
                        num_thresholds=127)
    writer.close()

In [65]:
import torch
dummy = torch.ones((10, 750, 48))

In [66]:
from resnet1d import ECGResNet50
model = ECGResNet50(750, 27)
print(model)

ECGResNet50(
  (encoder): ResNetEncoder(
    (gate): Sequential(
      (0): Conv1d(750, 64, kernel_size=(9,), stride=(1,), bias=False)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    )
    (blocks): ModuleList(
      (0): ResNetLayer(
        (blocks): Sequential(
          (0): ResNet753Block(
            (blocks): Sequential(
              (0): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (1): ReLU(inplace=True)
              (2): Sequential(
                (0): Conv1dAuto(64, 64, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
                (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [67]:
model(dummy)

tensor([[ 0.0319,  0.0396,  0.0684, -0.0213,  0.0171,  0.0258, -0.0172, -0.0036,
          0.0153,  0.0287,  0.0473,  0.0518, -0.0463, -0.0531,  0.0248, -0.0358,
          0.0298,  0.0782,  0.0598,  0.0756, -0.0384,  0.0456, -0.0704, -0.0812,
         -0.0573, -0.0322,  0.0258],
        [ 0.0319,  0.0396,  0.0684, -0.0213,  0.0171,  0.0258, -0.0172, -0.0036,
          0.0153,  0.0287,  0.0473,  0.0518, -0.0463, -0.0531,  0.0248, -0.0358,
          0.0298,  0.0782,  0.0598,  0.0756, -0.0384,  0.0456, -0.0704, -0.0812,
         -0.0573, -0.0322,  0.0258],
        [ 0.0319,  0.0396,  0.0684, -0.0213,  0.0171,  0.0258, -0.0172, -0.0036,
          0.0153,  0.0287,  0.0473,  0.0518, -0.0463, -0.0531,  0.0248, -0.0358,
          0.0298,  0.0782,  0.0598,  0.0756, -0.0384,  0.0456, -0.0704, -0.0812,
         -0.0573, -0.0322,  0.0258],
        [ 0.0319,  0.0396,  0.0684, -0.0213,  0.0171,  0.0258, -0.0172, -0.0036,
          0.0153,  0.0287,  0.0473,  0.0518, -0.0463, -0.0531,  0.0248, -0.0358

In [76]:
from torchvision import datasets, models, transforms
from myeval import agg_y_preds_bags, binary_acc, geometry_loss
import torch.optim as optim
from torch.optim import lr_scheduler
from snippets.pytorchtools import EarlyStopping
from sklearn.model_selection import GroupKFold
import time
from imbalanced_sampler import ImbalancedDatasetSampler, WeightedImbalancedDatasetSampler
st = time.time()
patience = 50
batch_size= 512#65000

saved_dir = '../saved/ECGResNet50/'
model.to(device)


trainDataset = torch.utils.data.Subset(signal_datasets_train, range(0,len(Signals_train), 1))
testDataset = torch.utils.data.Subset(signal_datasets_test, range(0,len(Signals_test), 1))

trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batch_size, pin_memory=True, 
                                       sampler=WeightedImbalancedDatasetSampler(trainDataset, weights=sample_weights))
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=65000, shuffle = False, pin_memory=True)




In [77]:
pos_weight = np.ones(N_CLASS) * 5
pos_weight = torch.Tensor(pos_weight).to(device)


#criterion_train = WeightedBCELoss(weights=weights_tensor, reduction='mean')
#criterion_test = WeightedBCELoss(weights=weights_tensor, reduction='mean')

class_weights_train = 1.0/np.sum(Data_labels_train, axis=0)
class_weights_train = class_weights_train / np.sum(class_weights_train)
class_weights_train = torch.Tensor(class_weights_train).to(device)
criterion_train = nn.BCEWithLogitsLoss(reduction='mean') 
criterion_test = nn.BCEWithLogitsLoss(reduction='mean') 

losses_train = []
losses_test = []

avg_losses_train = []
avg_losses_test = []



In [78]:
early_stopping = EarlyStopping(patience, verbose=False, 
                              saved_dir=saved_dir, 
                              save_name='resnet50_'+run_name)

In [79]:
optimizer = optim.Adam(model.parameters(), lr=0.01) 
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) #
# Decay LR by a factor of 0.1 every 100 epochs
#scheduler = lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=20, mode='max')

In [80]:
names = [get_name(label, Dx_map, Dx_map_unscored) for label in labels]

assert len(labels) == 27

In [81]:
y_trains_tensor = None
y_tests_tensor = None
for epoch in range(0, 5000):

    model.train()

    running_loss_train = 0.0

    y_trains = [] # ground truth
    output_trains = [] # output
    for k, (X_train, y_train) in tqdm(enumerate(trainLoader)):
        y_train = y_train.to(device)
        X_train = X_train.to(device)
        optimizer.zero_grad()
        output_train = model(X_train)
        output_trains.append(output_train.cpu())
        
        loss_train = criterion_train(output_train, y_train)
        losses_train.append(loss_train.item())
        
        avg_loss_train = np.average(losses_train)
        avg_losses_train.append(avg_loss_train)
    
        
        if np.mod(k, 100) == 0:
            writer.add_scalar('training loss',
            avg_loss_train,
            epoch * (len(Data_labels_train)//batch_size//100+1) + k//100)
        
        y_trains.append(y_train.cpu())
            
        loss_train.backward()
        optimizer.step()
            
    #scheduler.step()
    

    y_tests = [] # ground truth
    output_tests = [] # output
    with torch.no_grad():
        model.eval()
        
        for X_test, y_test in testLoader:  
            y_test = y_test.to(device)
            X_test = X_test.to(device)
            output_test = model(X_test)

            loss_test = criterion_test(output_test, y_test)
            losses_test.append(loss_test.item())

            output_tests.append(output_test.cpu())
            y_tests.append(y_test.cpu())
            
            
            
        avg_loss_test = np.average(losses_test)
        avg_losses_test.append(avg_loss_test)

        writer.add_scalar('testing loss',
                avg_loss_test,
                epoch)



    y_trains_tensor = torch.cat(y_trains, axis=0) # ground truth
    y_tests_tensor = torch.cat(y_tests, axis=0) # ground truth

    output_trains = torch.cat(output_trains, axis=0) 
    y_train_preds = torch.sigmoid(output_trains)

    output_tests = torch.cat(output_tests, axis=0)
    y_test_preds = torch.sigmoid(output_tests)

    #output_trains = torch.cat(output_trains, axis=0)
#     y_train_preds_max, y_train_preds_mean, _ = agg_y_preds_bags(y_train_preds, bag_size=n_segments)
#     y_test_preds_max, y_test_preds_mean, _ = agg_y_preds_bags(y_test_preds, bag_size=n_segments)
#     _, _, y_trains = agg_y_preds_bags(y_trains, bag_size=n_segments)
#     _, _, y_tests = agg_y_preds_bags(y_tests, bag_size=n_segments)


    for class_idx in range(N_CLASS):
        add_pr_curve_tensorboard(class_idx, y_trains_tensor, y_train_preds, names, global_step=epoch, prefix='train')
        add_pr_curve_tensorboard(class_idx, y_tests_tensor, y_test_preds, names, global_step=epoch, prefix='test')



    acc, fmeasure, fbeta, gbeta = binary_acc(y_train_preds, y_trains_tensor)           
    acc2, fmeasure2, fbeta2, gbeta2 = binary_acc(y_test_preds, y_tests_tensor)
    geometry = geometry_loss(fbeta, gbeta)
    geometry2 = geometry_loss(fbeta2, gbeta2)
    
#     output_str = 'S{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
#         epoch, (time.time()-st)/60,
#         avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry, np.nan,
#         avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2, np.nan)
#     print(output_str)
    score = compute_score(np.round(y_train_preds.data.numpy()), np.round(y_trains_tensor.data.numpy()), weights)
    score2 = compute_score(np.round(y_test_preds.data.numpy()), np.round(y_tests_tensor.data.numpy()), weights)
    output_str = 'S{} {:.2f} min |\n Train Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f} |\n Valid Loss: {:.6f}, Acc: {:.3f}, F: {:.3f}, Fbeta: {:.3f}, gbeta: {:.3f}, geo: {:.3f}, score: {:.3f}\n '.format(
        epoch, (time.time()-st)/60,
        avg_loss_train, acc, fmeasure, fbeta, gbeta, geometry, score,
        avg_loss_test, acc2, fmeasure2, fbeta2, gbeta2, geometry2, score2)
    scheduler.step(score2)
    
    writer.add_scalar('training score',
            score,
            epoch)

    writer.add_scalar('testing score',
            score2,
            epoch)
    
    print(output_str)

    with open(saved_dir+'loss_{}.txt'.format(run_name), 'a') as f:
        print(output_str, file=f)

    early_stopping(avg_loss_test, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break


#     output_string = 'AUROC|AUPRC|Accuracy|F-measure|Fbeta-measure|Gbeta-measure|Geomotry\n{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}|{:.3f}'.format(auroc2,auprc2,acc2,fmeasure2,fbeta2,gbeta2,geometry2)
#     print(output_string)     
#     with open(saved_dir+'score'+ str(i)+ '_epoch' + str(epoch) + '.txt', 'w') as f:
#         f.write(output_string)

#     avg_losses_train = np.array(avg_losses_train)
#     avg_losses_test = np.array(avg_losses_test)

#     np.save(saved_dir + 'avg_losses_train' + str(i) + '_epoch' + str(epoch), avg_losses_train)
#     np.save(saved_dir + 'avg_losses_test' + str(i) + '_epoch' + str(epoch), avg_losses_test)



S0 4.03 min |
 Train Loss: 0.142190, Acc: 0.954, F: 0.131, Fbeta: 0.155, gbeta: 0.070, geo: 0.104, score: -0.018 |
 Valid Loss: 0.166484, Acc: 0.953, F: 0.115, Fbeta: 0.138, gbeta: 0.060, geo: 0.091, score: -0.027
 



S1 7.80 min |
 Train Loss: 0.140277, Acc: 0.955, F: 0.146, Fbeta: 0.171, gbeta: 0.079, geo: 0.116, score: 0.076 |
 Valid Loss: 0.164823, Acc: 0.953, F: 0.120, Fbeta: 0.146, gbeta: 0.062, geo: 0.095, score: 0.168
 



S2 11.56 min |
 Train Loss: 0.138861, Acc: 0.955, F: 0.154, Fbeta: 0.180, gbeta: 0.084, geo: 0.123, score: 0.130 |
 Valid Loss: 0.164377, Acc: 0.953, F: 0.147, Fbeta: 0.170, gbeta: 0.078, geo: 0.115, score: 0.138
 



S3 15.39 min |
 Train Loss: 0.137626, Acc: 0.956, F: 0.161, Fbeta: 0.187, gbeta: 0.088, geo: 0.129, score: 0.147 |
 Valid Loss: 0.163581, Acc: 0.954, F: 0.136, Fbeta: 0.160, gbeta: 0.071, geo: 0.107, score: 0.174
 



S4 19.19 min |
 Train Loss: 0.136741, Acc: 0.956, F: 0.165, Fbeta: 0.191, gbeta: 0.091, geo: 0.132, score: 0.161 |
 Valid Loss: 0.163680, Acc: 0.954, F: 0.151, Fbeta: 0.172, gbeta: 0.080, geo: 0.117, score: 0.021
 


...


S32 126.70 min |
 Train Loss: 0.125778, Acc: 0.960, F: 0.240, Fbeta: 0.283, gbeta: 0.136, geo: 0.196, score: 0.343 |
 Valid Loss: 0.164801, Acc: 0.954, F: 0.174, Fbeta: 0.201, gbeta: 0.091, geo: 0.135, score: 0.165
 



S33 130.73 min |
 Train Loss: 0.125551, Acc: 0.960, F: 0.243, Fbeta: 0.287, gbeta: 0.138, geo: 0.199, score: 0.348 |
 Valid Loss: 0.165031, Acc: 0.954, F: 0.165, Fbeta: 0.191, gbeta: 0.087, geo: 0.129, score: 0.093
 



S34 134.63 min |
 Train Loss: 0.125329, Acc: 0.960, F: 0.243, Fbeta: 0.287, gbeta: 0.138, geo: 0.199, score: 0.348 |
 Valid Loss: 0.165103, Acc: 0.954, F: 0.164, Fbeta: 0.191, gbeta: 0.086, geo: 0.128, score: 0.106
 



S35 138.51 min |
 Train Loss: 0.125051, Acc: 0.961, F: 0.255, Fbeta: 0.302, gbeta: 0.146, geo: 0.210, score: 0.378 |
 Valid Loss: 0.165338, Acc: 0.954, F: 0.176, Fbeta: 0.206, gbeta: 0.092, geo: 0.138, score: 0.150
 


KeyboardInterrupt: 

In [ ]:
len(Signals_test)

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix
from global_vars import labels

cf_matrices = multilabel_confusion_matrix(y_trains_tensor.data.numpy(), np.round(y_train_preds.data.numpy()))

In [ ]:
for label, cf_matrix in zip(labels, cf_matrices):
    print(get_name(label, Dx_map, Dx_map_unscored))
    print(cf_matrix)
    print('\n')


In [ ]:
y_trains_tensor.data.numpy()